In [1]:
import cv2
import mediapipe as mp
import numpy as np

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as ax3d
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [7]:
cap = cv2.VideoCapture('http://192.168.0.40:8000')
while True:
    ret, frame = cap.read()
    cv2.imshow('Mediapipe', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.4-dev) /Users/runner/work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:1006: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [ ]:
%matplotlib tk

# LINES: Body(2), L_ARM(4), R_ARM(4), L_LEG(4), R_LEG(4)
LINES = [[11, 12], [23, 24], [11, 23], [12, 24],
            [11, 13], [13, 15], [15, 19], [15, 17], [17, 19],
            [12, 14], [14, 16], [16, 20], [16, 18], [18, 20],
            [23, 25], [25, 27], [27, 29], [29, 31], [27, 31],
            [24, 26], [26, 28], [28, 30], [30, 32], [28, 32]]
POINTS = [1, 4, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

def plot_fencing_3d(fig, pose):
    ax = fig.gca(projection='3d')
    
    min_val = -1.0
    max_val = 1.0
    ax.set_xlim3d(min_val, max_val)
    ax.set_ylim3d(min_val, max_val)
    ax.set_zlim3d(min_val, max_val)
    
    for line in LINES:
        line_color = '#2ca02c'
        if line[0] == 25 or line[0] == 13:
            line_color = '#d62728'
        ax.plot([pose[line[0]].x, pose[line[1]].x], [pose[line[0]].z, pose[line[1]].z],
                   [-pose[line[0]].y, -pose[line[1]].y], color=line_color)
        
    '''
    # Draw Backbone
    bottom_center = [(pose[23].x + pose[24].x)/2, (pose[23].z + pose[24].z)/2, -(pose[23].y + pose[24].y)/2]
    ax.plot([shoulder_center[0], bottom_center[0]], [shoulder_center[1], bottom_center[1]], 
            [shoulder_center[2], bottom_center[2]], color='#2ca02c')
    '''
    
    '''
    for point in POINTS:
        point_color = '#1f77b4'
        ax.scatter(pose[point].x, pose[point].y, pose[point].z, color=point_color, marker='o', edgecolor=point_color)
    '''
    
    # Draw Head
    shoulder_center = [(pose[11].x + pose[12].x)/2, (pose[11].z + pose[12].z)/2, -(pose[11].y + pose[12].y)/2]
    bottom_center = [(pose[23].x + pose[24].x)/2, (pose[23].z + pose[24].z)/2, -(pose[23].y + pose[24].y)/2]
    
    ax.plot([shoulder_center[0], 1.5*shoulder_center[0] - 0.5*bottom_center[0]], 
            [shoulder_center[1], 1.5*shoulder_center[1] - 0.5*bottom_center[1]], 
            [shoulder_center[2], 1.5*shoulder_center[2] - 0.5*bottom_center[2]], color='#1f77b4')

In [ ]:
cap = cv2.VideoCapture('fencing_full_1.mov')

fig = plt.figure(figsize=(8,8))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    poses = []
    i = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Make detection
        results = pose.process(frame)
        
        
        # Render detection
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
        
        #cv2.imshow('Fencing Feed', frame)
        
        
        if(results.pose_world_landmarks != None):
            poses.append(results.pose_world_landmarks.landmark)
            i += 1
        
        if i > 500:
            break;
        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Make detection
        results = pose.process(frame)
        
        # Render detection
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
        
        cv2.imshow('Webcam', frame)
        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
for pose in poses:
    plot_fencing_3d(fig, pose)
    plt.pause(0.0001)
    plt.cla()
    #fencing_3d_animate(poses)